# CMMA Threshold Heatmap

This notebook creates heatmaps for CMMA threshold parameter testing, with a fallback to matplotlib if seaborn is not available.

In [ ]:
import sys
import os

# Import our backtester package
sys.path.append(os.path.abspath('../../'))
from backtester import get_price_data, get_vwap
print("Using backtester package")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn, but don't fail if it's not available
try:
    import seaborn as sns
    print(f"Seaborn version: {sns.__version__}")
    HAS_SEABORN = True
except ImportError:
    print("Seaborn not available, using matplotlib instead")
    HAS_SEABORN = False

## 1. Load Data

First, let's load the Squid_Ink price data and limit it to the first 20,000 timestamps (in-sample data).

In [ ]:
# Load data directly using backtester package
print("Loading price data...")
prices = get_price_data('SQUID_INK', 1)
print(f"Loaded {len(prices)} price data points")

# Limit to first 20,000 timestamps (in-sample data)
in_sample_prices = prices.iloc[:20000]
print(f"Limited to {len(in_sample_prices)} in-sample data points")

# Get VWAP
print("Getting VWAP for SQUID_INK...")
squid_vwap = in_sample_prices['vwap']
print(f"Got VWAP with {len(squid_vwap)} data points")
print(f"VWAP range: {squid_vwap.min()} to {squid_vwap.max()}")

# Calculate returns
returns = squid_vwap.pct_change().dropna()
print(f"Calculated returns with {len(returns)} data points")

## 2. Calculate CMMA (Cumulative Moving Average Momentum)

Let's calculate the CMMA indicator using the same calculation as in the momentum strategy.

In [ ]:
def calculate_cmma(prices, lookback=10):
    """
    Compute the Cumulative Moving Average Momentum (CMMA).
    
    Parameters:
        prices (pd.Series): Series of prices
        lookback (int): Lookback period
        
    Returns:
        pd.Series: CMMA indicator (0-1 range)
    """
    # Calculate raw CMMA
    raw_cmma = (prices - prices.rolling(lookback).mean().shift(1)).divide(np.sqrt(lookback+1)).dropna()
    
    # Normalize using sigmoid function
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    cmma = sigmoid(raw_cmma)
    return cmma

# Calculate CMMA with a fixed lookback period
lookback = 20  # Use a fixed lookback period
cmma = calculate_cmma(squid_vwap, lookback)

# Display the first few rows
cmma.head(10)

## 3. Define Filtered CMMA Mean Reversion Strategy

Let's define a strategy that only buys below 2000 and only sells above 2000, using CMMA as a filter.

In [ ]:
def filtered_cmma_strategy(prices, cmma, fair_price, upper_threshold=0.7, lower_threshold=0.3, holding_period=10):
    """
    Implement a filtered CMMA-based mean reversion strategy.
    
    Parameters:
        prices (pd.Series): Series of prices
        cmma (pd.Series): CMMA indicator
        fair_price (float): Fair price to revert to
        upper_threshold (float): Upper threshold for CMMA
        lower_threshold (float): Lower threshold for CMMA
        holding_period (int): Number of periods to hold the position
        
    Returns:
        pd.Series: Portfolio positions (1 for long, -1 for short, 0 for no position)
    """
    # Initialize positions
    positions = pd.Series(0, index=prices.index)
    
    # Get valid indices where CMMA is not NaN
    valid_indices = cmma.dropna().index
    
    # Set positions based on CMMA and fair price
    for time in valid_indices:
        # Get the current price and CMMA value
        current_price = prices.loc[time]
        current_cmma = cmma.loc[time]
        
        # Get the index position
        idx = prices.index.get_loc(time)
        
        # ONLY short above fair price when CMMA is high (strong momentum)
        if current_price > fair_price and current_cmma > upper_threshold:
            # Set short position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = -1
        
        # ONLY buy below fair price when CMMA is low (weak momentum)
        elif current_price < fair_price and current_cmma < lower_threshold:
            # Set long position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = 1
    
    return positions

## 4. Test CMMA Threshold Parameters

Let's test different upper and lower threshold parameters for the CMMA indicator.

In [ ]:
# Define the fair price
FAIR_PRICE = 2000

# Define parameters for testing
upper_thresholds = [0.6, 0.65, 0.7, 0.75, 0.8]
lower_thresholds = [0.2, 0.25, 0.3, 0.35, 0.4]
holding_period = 10  # Use a fixed holding period for threshold testing

# Initialize results dictionary
threshold_results = {}

# Test different threshold combinations
for upper in upper_thresholds:
    for lower in lower_thresholds:
        # Skip invalid combinations (upper <= lower)
        if upper <= lower:
            continue
            
        strategy_name = f'upper_{upper}_lower_{lower}'
        
        # Get positions
        positions = filtered_cmma_strategy(squid_vwap, cmma, FAIR_PRICE, upper, lower, holding_period)
        
        # Calculate strategy returns
        strategy_returns = positions.shift(1) * returns
        strategy_returns = strategy_returns.dropna()
        
        # Calculate performance metrics
        total_return = strategy_returns.sum()
        sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # Annualized
        win_rate = (strategy_returns > 0).mean()
        
        # Count the number of trades
        num_trades = (positions.diff() != 0).sum()
        
        # Store results
        threshold_results[strategy_name] = {
            'Upper Threshold': upper,
            'Lower Threshold': lower,
            'Total Return': total_return,
            'Sharpe Ratio': sharpe_ratio,
            'Win Rate': win_rate,
            'Number of Trades': num_trades
        }

# Convert results to DataFrame
threshold_df = pd.DataFrame(threshold_results).T

# Sort by Sharpe Ratio
threshold_df = threshold_df.sort_values('Sharpe Ratio', ascending=False)

# Display top 10 results
threshold_df.head(10)

## 5. Create Heatmaps for Threshold Parameters

Let's create heatmaps to visualize the results of the threshold parameter testing.

In [ ]:
# Create a pivot table for heatmap visualization
pivot_data = threshold_df.reset_index()
sharpe_pivot = pd.pivot_table(
    pivot_data, 
    values='Sharpe Ratio', 
    index='Upper Threshold', 
    columns='Lower Threshold'
)

trades_pivot = pd.pivot_table(
    pivot_data, 
    values='Number of Trades', 
    index='Upper Threshold', 
    columns='Lower Threshold'
)

# Create heatmaps
if HAS_SEABORN:
    # Use seaborn for heatmaps if available
    plt.figure(figsize=(12, 8))
    sns.heatmap(sharpe_pivot, annot=True, cmap='viridis', fmt='.2f')
    plt.title('Sharpe Ratio by CMMA Threshold Parameters')
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(trades_pivot, annot=True, cmap='Blues', fmt='d')
    plt.title('Number of Trades by CMMA Threshold Parameters')
    plt.tight_layout()
    plt.show()
else:
    # Use matplotlib for heatmaps if seaborn is not available
    plt.figure(figsize=(12, 8))
    plt.imshow(sharpe_pivot, cmap='viridis', interpolation='nearest')
    plt.colorbar(label='Sharpe Ratio')
    
    # Add labels
    plt.title('Sharpe Ratio by CMMA Threshold Parameters')
    plt.xlabel('Lower Threshold')
    plt.ylabel('Upper Threshold')
    
    # Add tick labels
    plt.xticks(range(len(sharpe_pivot.columns)), sharpe_pivot.columns)
    plt.yticks(range(len(sharpe_pivot.index)), sharpe_pivot.index)
    
    # Add text annotations
    for i in range(len(sharpe_pivot.index)):
        for j in range(len(sharpe_pivot.columns)):
            if not np.isnan(sharpe_pivot.iloc[i, j]):
                plt.text(j, i, f'{sharpe_pivot.iloc[i, j]:.2f}', 
                         ha='center', va='center', color='white')
    
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(12, 8))
    plt.imshow(trades_pivot, cmap='Blues', interpolation='nearest')
    plt.colorbar(label='Number of Trades')
    
    # Add labels
    plt.title('Number of Trades by CMMA Threshold Parameters')
    plt.xlabel('Lower Threshold')
    plt.ylabel('Upper Threshold')
    
    # Add tick labels
    plt.xticks(range(len(trades_pivot.columns)), trades_pivot.columns)
    plt.yticks(range(len(trades_pivot.index)), trades_pivot.index)
    
    # Add text annotations
    for i in range(len(trades_pivot.index)):
        for j in range(len(trades_pivot.columns)):
            if not np.isnan(trades_pivot.iloc[i, j]):
                plt.text(j, i, f'{int(trades_pivot.iloc[i, j])}', 
                         ha='center', va='center', color='white')
    
    plt.tight_layout()
    plt.show()

## 6. Analyze Best Threshold Parameters

Let's analyze the performance of the strategy with the best threshold parameters.

In [ ]:
# Get the best threshold parameters
best_strategy = threshold_df.index[0]
best_upper = threshold_df.loc[best_strategy, 'Upper Threshold']
best_lower = threshold_df.loc[best_strategy, 'Lower Threshold']

print(f'Best threshold parameters: Upper = {best_upper}, Lower = {best_lower}')

# Get positions for the best threshold parameters
best_positions = filtered_cmma_strategy(squid_vwap, cmma, FAIR_PRICE, best_upper, best_lower, holding_period)

# Calculate strategy returns
best_returns = best_positions.shift(1) * returns
best_returns = best_returns.dropna()

# Plot cumulative returns
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(best_returns.cumsum(), label=f'Strategy Returns (Upper={best_upper}, Lower={best_lower})')
plt.title(f'Cumulative Returns of Best CMMA-Filtered Strategy')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(squid_vwap, label='VWAP')
plt.axhline(y=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')

# Plot buy and sell signals
buy_signals = best_positions.diff() > 0
sell_signals = best_positions.diff() < 0

plt.scatter(buy_signals[buy_signals].index, squid_vwap[buy_signals], 
            marker='^', s=100, color='green', label='Buy Signal (Below 2000)')
plt.scatter(sell_signals[sell_signals].index, squid_vwap[sell_signals], 
            marker='v', s=100, color='red', label='Sell Signal (Above 2000)')

plt.title('Price with Trading Signals')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Conclusion

In this notebook, we've explored the impact of different threshold parameters on a CMMA-filtered mean reversion strategy for Squid_Ink. The strategy only buys below 2000 when CMMA is low (weak momentum) and only sells above 2000 when CMMA is high (strong momentum).

Key findings:
1. The optimal threshold parameters are [to be filled after running]
2. Higher upper thresholds and lower lower thresholds generally [increase/decrease] the selectivity of the strategy
3. More selective strategies (higher upper, lower lower) tend to have [higher/lower] Sharpe ratios but [fewer/more] trades
4. The best threshold combination balances the trade-off between signal quality and signal frequency

These findings suggest that the threshold parameters are critical for the CMMA-filtered mean reversion strategy. The optimal parameters ensure that we only take high-quality mean reversion trades when the CMMA indicator suggests a potential reversal.